In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os
import operator

import jsonlines
import pandas as pd
import pyspark
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.clustering import KMeans, GaussianMixture

In [8]:
spark = (
    pyspark.sql.SparkSession
    .builder
    .appName("Python Spark cluster analysis")
    .enableHiveSupport()
    .getOrCreate()
)

In [9]:
path_aggregated_df = "../data/output/k_means/clustered_kmeans__k_5_parquet/"

In [10]:
clustering_df = spark.read.parquet(path_aggregated_df)

clustering_df

DataFrame[user_account_id: int, _c0: double, user_lifetime: double, user_no_outgoing_activity_in_days: double, user_account_balance_last: double, user_spendings: double, reloads_inactive_days: double, reloads_count: double, reloads_sum: double, calls_outgoing_count: double, calls_outgoing_spendings: double, calls_outgoing_duration: double, calls_outgoing_spendings_max: double, calls_outgoing_duration_max: double, calls_outgoing_inactive_days: double, calls_outgoing_to_onnet_count: double, calls_outgoing_to_onnet_spendings: double, calls_outgoing_to_onnet_duration: double, calls_outgoing_to_onnet_inactive_days: double, calls_outgoing_to_offnet_count: double, calls_outgoing_to_offnet_spendings: double, calls_outgoing_to_offnet_duration: double, calls_outgoing_to_offnet_inactive_days: double, calls_outgoing_to_abroad_count: double, calls_outgoing_to_abroad_spendings: double, calls_outgoing_to_abroad_duration: double, calls_outgoing_to_abroad_inactive_days: double, sms_outgoing_count: doub

In [11]:
df = clustering_df.toPandas()
df.head(5)

,user_account_id,_c0,user_lifetime,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,reloads_inactive_days,reloads_count,reloads_sum,calls_outgoing_count,...,user_has_outgoing_sms,user_use_gprs,user_does_reload,n_months,year,month,churn,initial_features,features,prediction
0,985904,43383.666667,438.0,0.666667,1.826667,2.616667,19.333333,1.000000,2.633333,327.666667,...,1,0,1,3,2013,9,0,"[438.0, 327.6666666666667, 0.6666666666666666,...","[-0.6034185315836667, 7.015670480484471, -0.40...",3
1,445287,43581.666667,617.0,2.333333,0.366667,25.026667,15.000000,4.000000,25.030000,106.666667,...,1,1,1,3,2013,9,1,"[617.0, 106.66666666666669, 2.333333333333333,...","[-0.5732645094670495, 1.8461144918771961, -0.4...",3
2,994907,43611.666667,451.0,7.000000,0.706667,3.330000,40.000000,0.666667,3.333333,55.000000,...,1,0,1,3,2013,9,1,"[451.0, 55.0, 7.0, 2.69, 40.0, 0.6666666666666...","[-0.6012285746701693, 0.63754559559646, -0.393...",0
3,1129137,43648.666667,5333.0,15.000000,0.000000,5.000000,53.000000,0.333333,5.000000,51.000000,...,1,0,1,3,2013,9,1,"[5333.0, 51.0, 15.0, 3.953333333333333, 53.0, ...","[0.2211844754601365, 0.543978971368274, -0.375...",0
4,339124,43764.333333,1361.0,0.666667,2.373333,2.003333,53.000000,0.333333,2.000000,96.666667,...,0,0,1,3,2013,9,0,"[1361.0, 96.66666666666669, 0.6666666666666666...","[-0.44793159072535604, 1.6121979313067312, -0....",0


In [12]:
df.describe(include="all")

,user_account_id,_c0,user_lifetime,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,reloads_inactive_days,reloads_count,reloads_sum,calls_outgoing_count,...,user_has_outgoing_sms,user_use_gprs,user_does_reload,n_months,year,month,churn,initial_features,features,prediction
count,3.880600e+04,38806.000000,38806.000000,38806.000000,38806.000000,38806.000000,38806.000000,38806.000000,38806.000000,38806.000000,...,38806.000000,38806.000000,38806.000000,38806.000000,38806.0,38806.0,38806.000000,38806,38806,38806.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33798,33798,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[15947.0, 0.0, 1338.0, 0.0, 8.0, 1.0, 0.0, 8.0...","[2.0092000662186504, -0.6489954875410973, 2.70...",NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2420,2420,NaN
mean,6.088175e+05,73300.502843,4020.006962,12.599753,7.681185,4.171476,380.971409,0.706480,4.511054,27.744743,...,0.674767,0.049065,0.717544,2.680230,2013.0,9.0,0.223058,NaN,NaN,1.031155
std,4.355919e+05,21336.541021,5936.189849,53.853449,8.253671,5.707491,514.295734,0.780139,6.253145,42.750287,...,0.468468,0.216006,0.450200,0.727875,0.0,0.0,0.416302,NaN,NaN,1.125719
min,1.300000e+01,43235.666667,1.000000,0.000000,0.000000,-0.083333,0.000000,0.000000,-6.666667,0.000000,...,0.000000,0.000000,0.000000,1.000000,2013.0,9.0,0.000000,NaN,NaN,0.000000
25%,2.574325e+05,56826.916667,518.250000,1.000000,0.500000,0.000000,16.000000,0.000000,0.000000,3.333333,...,0.000000,0.000000,0.000000,3.000000,2013.0,9.0,0.000000,NaN,NaN,0.000000
50%,4.168255e+05,70371.833333,1282.000000,2.666667,5.346667,2.086667,37.000000,0.666667,2.000000,14.000000,...,1.000000,0.000000,1.000000,3.000000,2013.0,9.0,0.000000,NaN,NaN,1.000000
75%,1.106625e+06,83826.083333,2017.000000,8.000000,11.536667,5.880000,864.000000,1.000000,7.333333,35.333333,...,1.000000,0.000000,1.000000,3.000000,2013.0,9.0,0.000000,NaN,NaN,2.000000


In [13]:
grouped_df = df.groupby('prediction').describe(include="all")

In [14]:
avg_df = grouped_df['user_account_id']
avg_df['pcc'] = avg_df['count'] / avg_df['count'].sum()
avg_df = avg_df[['count', 'pcc']]

excluded = ['prediction', 'initial_features', 'features', 'user_account_id', '_c0', 'year', 'month']
for col in df.columns:  
    if not col in excluded:
        avg_df[col] = grouped_df[col]['mean']

/home/majo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
avg_df

,count,pcc,user_lifetime,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,reloads_inactive_days,reloads_count,reloads_sum,calls_outgoing_count,...,last_100_sms_outgoing_to_offnet_count,last_100_sms_outgoing_to_abroad_count,last_100_gprs_usage,user_intake,user_has_outgoing_calls,user_has_outgoing_sms,user_use_gprs,user_does_reload,n_months,churn
prediction,,,,,,,,,,,,,,,,,,,,,
0,18499.0,0.476705,1810.727607,3.136233,10.021976,4.713373,104.389165,0.720291,5.526329,28.872588,...,49.960944,2.362443,0.623459,0.077896,0.998649,0.886751,0.065463,0.937186,2.886318,0.040056
1,5485.0,0.141344,15914.724430,8.350319,0.322341,0.075654,34.996536,0.950927,0.382368,0.888453,...,0.268672,0.006442,0.004617,0.040656,0.202552,0.032999,0.002917,0.947675,1.160073,0.863446
2,10553.0,0.271942,2392.291023,10.840898,7.515656,0.943281,1142.279841,0.044632,0.435377,9.803468,...,12.399760,0.659497,0.872694,0.016299,0.991851,0.549322,0.026627,0.125272,2.998958,0.266938
3,3652.0,0.094109,1168.265745,1.438299,7.854073,17.208093,21.047691,2.240827,17.460562,116.166849,...,60.186017,2.611948,0.742978,0.066265,1.000000,0.950986,0.102683,0.997262,2.944414,0.024096
4,617.0,0.015900,9236.828201,430.260400,4.725575,2.386164,858.314425,0.357645,3.835710,16.171799,...,7.990816,0.259860,0.036645,0.513776,0.995138,0.534846,0.034036,0.560778,3.000000,0.444084


In [16]:
avg_df

,count,pcc,user_lifetime,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,reloads_inactive_days,reloads_count,reloads_sum,calls_outgoing_count,...,last_100_sms_outgoing_to_offnet_count,last_100_sms_outgoing_to_abroad_count,last_100_gprs_usage,user_intake,user_has_outgoing_calls,user_has_outgoing_sms,user_use_gprs,user_does_reload,n_months,churn
prediction,,,,,,,,,,,,,,,,,,,,,
0,18499.0,0.476705,1810.727607,3.136233,10.021976,4.713373,104.389165,0.720291,5.526329,28.872588,...,49.960944,2.362443,0.623459,0.077896,0.998649,0.886751,0.065463,0.937186,2.886318,0.040056
1,5485.0,0.141344,15914.724430,8.350319,0.322341,0.075654,34.996536,0.950927,0.382368,0.888453,...,0.268672,0.006442,0.004617,0.040656,0.202552,0.032999,0.002917,0.947675,1.160073,0.863446
2,10553.0,0.271942,2392.291023,10.840898,7.515656,0.943281,1142.279841,0.044632,0.435377,9.803468,...,12.399760,0.659497,0.872694,0.016299,0.991851,0.549322,0.026627,0.125272,2.998958,0.266938
3,3652.0,0.094109,1168.265745,1.438299,7.854073,17.208093,21.047691,2.240827,17.460562,116.166849,...,60.186017,2.611948,0.742978,0.066265,1.000000,0.950986,0.102683,0.997262,2.944414,0.024096
4,617.0,0.015900,9236.828201,430.260400,4.725575,2.386164,858.314425,0.357645,3.835710,16.171799,...,7.990816,0.259860,0.036645,0.513776,0.995138,0.534846,0.034036,0.560778,3.000000,0.444084


In [17]:
avg_df.to_csv('../data/output/cluster_report_avg.csv')